## Code to find out the analytics in the Udacity enrollment
### First read the enrollments file and store it in a dictionary format 

In [1]:
import unicodecsv

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = read_csv('enrollments.csv')
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': u'2015-01-14',
 u'days_to_cancel': u'65',
 u'is_canceled': u'True',
 u'is_udacity': u'True',
 u'join_date': u'2014-11-10',
 u'status': u'canceled'}

### Now read the daily_engagement.csv and project_submissions.csv

In [2]:
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')
    
print daily_engagement[0]
print project_submissions[0]

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Data Wrangling
### Convert the data into suitable data-types

In [3]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [4]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [5]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

In [6]:
# Finding the number of rows and unique students in each table
def rows_and_uniqueStudents(table):
    account_key = set()
    for row in table:
        account_key.add(row['account_key'])
    return len(table),account_key

#Observe that the 'account_key' label in other two tables is actually 'acct' in daily_engagement table
# Change 'acct' to 'account_key'
for engagement in daily_engagement:
    engagement['account_key'] = engagement.pop('acct')
    
enrollment_num_rows, enrollment_unique_students = rows_and_uniqueStudents(enrollments)
engagement_num_rows, engagement_unique_students = rows_and_uniqueStudents(daily_engagement)
submission_num_rows, submission_unique_students = rows_and_uniqueStudents(project_submissions)

In [7]:
#No. of total enrollments, No. of unique students
print 'Actual Enrollments =', enrollment_num_rows, '     Unique students =', len(enrollment_unique_students)

# The difference in these numbers is due to that some students might have enrolled and 
# cancelled their enrollment/re-enrolled later

Actual Enrollments = 1640      Unique students = 1302


### Analysing the enrollments of students

In [17]:
# Find out the engagements that are strange
count = 0
count_pre = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in engagement_unique_students:
        count += 1
        if enrollment['join_date']!=enrollment['cancel_date']:
            count_pre += 1
            print enrollment
print 'No.of students with no corresponding engagement data:',count
print 'No. of student with more than one registration day:',count_pre
#count_pre are the udacity test accounts. Observe 'is_udacity' = True . 

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}
No.of students with no corresponding engagement data: 71
No. of student with more than one registration day: 3


### Remove the Udacity Test accounts from the data

In [18]:
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [19]:
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [21]:
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


In [38]:
paid_students = {}
for enrollment in non_udacity_enrollments:
    if enrollment['days_to_cancel'] == None or enrollment['days_to_cancel']>7:
        if enrollment['account_key'] in paid_students.keys():
            if paid_students[enrollment['account_key']] < enrollment['join_date']:
                paid_students[enrollment['account_key']] = enrollment['join_date']
        else: paid_students[enrollment['account_key']] = enrollment['join_date']
len(paid_students)

995

In [40]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

engagement_paid = []
for i in paid_students.keys():
    for engagement in non_udacity_engagement:
        if engagement['account_key'] == i:
            if within_one_week(paid_students[i],engagement['utc_date']):
                engagement_paid.append(engagement)
len(engagement_paid)

21508